# CNN Classification with MR Dataset
<hr>

The __modus operandi__ for text classification is to use __word embedding__ for representing words and a Convolutional neural network to learn how to discriminate documents on classification problems. 

__Yoav Goldberg__ commented in _A Primer on Neural Network Models for Natural Language Processing, 2015._ :
> _The non-linearity of the network, as well as the ability to easily integrate pre-trained
word embeddings, often lead to superior classification accuracy._

He also commented in _Neural Network Methods for Natural Language Processing, 2017_ :
> ... _the CNN is in essence a feature-extracting architecture. ... . The CNNs layer's responsibility is to extract meaningful sub-structures that are useful for the overall prediction task at hand._

We will build a text classification model using CNN model on the Movie Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 1,2, 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2) constraint: 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MR/MR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10662, 3)


,sentence,label,split
0,"simplistic , silly and tedious .",0,train
1,"it 's so laddish and juvenile , only teenage b...",0,train
2,exploitative and largely devoid of the depth o...,0,train
3,garbus discards the potential for pathological...,0,train
4,a visually flashy but narratively opaque and e...,0,train
...,...,...,...
10657,both exuberantly romantic and serenely melanch...,1,train
10658,mazel tov to a film about a family 's joyous l...,1,train
10659,standing in the shadows of motown is the best ...,1,train
10660,it 's nice to see piscopo again after all thes...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10662 non-null  object
 1   label     10662 non-null  int32 
 2   split     10662 non-null  object
dtypes: int32(1), object(2)
memory usage: 208.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5331,5331
1,5331,5331


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'simplistic , silly and tedious .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

print("Example of sentence: ", sentences[8])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[8])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[8])

Example of sentence:  unfortunately the story and the actors are served with a hack script .
Into a sequence of int: [744, 2, 43, 4, 2, 225, 29, 1579, 15, 3, 2631, 151]
Into a padded sequence: [ 744    2   43    4    2  225   29 1579   15    3 2631  151    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


In [10]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
is 7
's 8
it 9
in 10
18760


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          5628000   
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 4900)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4900)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                49010     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0

In [13]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [14]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=15, verbose=2, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/15
192/192 - 25s - loss: 0.6915 - accuracy: 0.5132 - val_loss: 0.6785 - val_accuracy: 0.6429
Epoch 2/15
192/192 - 17s - loss: 0.5716 - accuracy: 0.7010 - val_loss: 0.4762 - val_accuracy: 0.7901
Epoch 3/15
192/192 - 16s - loss: 0.3083 - accuracy: 0.8794 - val_loss: 0.4740 - val_accuracy: 0.7901
Epoch 4/15
192/192 - 17s - loss: 0.1486 - accuracy: 0.9513 - val_loss: 0.5957 - val_accuracy: 0.7929
Epoch 5/15
192/192 - 17s - loss: 0.0934 - accuracy: 0.9723 - val_loss: 0.7627 - val_accuracy: 0.7835
Epoch 6/15
192/192 - 18s - loss: 0.0671 - accuracy: 0.9804 - val_loss: 0.9089 - val_accuracy: 0.7676
Epoch 7/15
192/192 - 18s - loss: 0.0551 - accuracy: 0.9822 - val_loss: 1.0379 - val_accuracy: 0.7676
Epoch 8/15
192/192 - 17s - loss: 0.0442 - accuracy: 0.9868 - val_loss: 1.1547 - val_accuracy: 0.7563
Epoch 9/15
192/192 - 17s - loss: 0.0401 - accuracy: 0.9877 - val_loss: 1.1494 - val_accuracy: 0.7685
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Ac

Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 76.92307829856873
Epoch 1/15
192/192 - 18s - loss: 0.6898 - accuracy: 0.5195 - val_loss: 0.6760 - val_accuracy: 0.6323
Epoch 2/15
192/192 - 17s - loss: 0.5703 - accuracy: 0.6919 - val_loss: 0.5400 - val_accuracy: 0.7158
Epoch 3/15
192/192 - 16s - loss: 0.3292 - accuracy: 0.8600 - val_loss: 0.5723 - val_accuracy: 0.7195
Epoch 4/15
192/192 - 16s - loss: 0.1773 - accuracy: 0.9262 - val_loss: 0.6722 - val_accuracy: 0.7411
Epoch 5/15
192/192 - 17s - loss: 0.1031 - accuracy: 0.9496 - val_loss: 0.9112 - val_accuracy: 0.7233
Epoch 6/15
192/192 - 16s - loss: 0.0820 - accuracy: 0.9553 - val_loss: 1.1181 - val_accuracy: 0.7233
Epoch 7/15
192/192 - 16s - loss: 0.0618 - accuracy: 0.9745 - val_loss: 1.2121 - val_accuracy: 0.7261
Epoch 8/15
192/192 - 16s - loss: 0.0543 - accuracy: 0.9793 - val_loss: 1.3178 - val_accuracy: 0.7205
Epoch 9/15
192/192 - 16s - loss: 0.0499 - accuracy: 0.9791 - val_loss: 1.42

Epoch 7/15
192/192 - 17s - loss: 0.0489 - accuracy: 0.9857 - val_loss: 1.2512 - val_accuracy: 0.7467
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 77.57973670959473
Epoch 1/15
192/192 - 18s - loss: 0.6885 - accuracy: 0.5335 - val_loss: 0.6587 - val_accuracy: 0.6745
Epoch 2/15
192/192 - 16s - loss: 0.5307 - accuracy: 0.7334 - val_loss: 0.4691 - val_accuracy: 0.7655
Epoch 3/15
192/192 - 16s - loss: 0.2873 - accuracy: 0.8898 - val_loss: 0.5187 - val_accuracy: 0.7702
Epoch 4/15
192/192 - 16s - loss: 0.1509 - accuracy: 0.9509 - val_loss: 0.6290 - val_accuracy: 0.7720
Epoch 5/15
192/192 - 16s - loss: 0.0868 - accuracy: 0.9744 - val_loss: 0.7817 - val_accuracy: 0.7702
Epoch 6/15
192/192 - 16s - loss: 0.0644 - accuracy: 0.9804 - val_loss: 0.9244 - val_accuracy: 0.7692
Epoch 7/15
192/192 - 16s - loss: 0.0476 - accuracy: 0.9850 - val_loss: 1.1628 - val_accuracy: 0.7655
Epoch 8/15
192/192 - 16s - loss: 0.0375 - accuracy: 0.9891 - val_loss: 1.18

Epoch 3/15
192/192 - 16s - loss: 0.3173 - accuracy: 0.8818 - val_loss: 0.5006 - val_accuracy: 0.7702
Epoch 4/15
192/192 - 16s - loss: 0.1335 - accuracy: 0.9566 - val_loss: 0.6822 - val_accuracy: 0.7664
Epoch 5/15
192/192 - 16s - loss: 0.0742 - accuracy: 0.9792 - val_loss: 0.8296 - val_accuracy: 0.7674
Epoch 6/15
192/192 - 16s - loss: 0.0529 - accuracy: 0.9849 - val_loss: 1.0872 - val_accuracy: 0.7570
Epoch 7/15
192/192 - 16s - loss: 0.0434 - accuracy: 0.9878 - val_loss: 1.1595 - val_accuracy: 0.7683
Epoch 8/15
192/192 - 16s - loss: 0.0378 - accuracy: 0.9899 - val_loss: 1.2989 - val_accuracy: 0.7533
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 77.01688408851624
Epoch 1/15
192/192 - 17s - loss: 0.6938 - accuracy: 0.5041 - val_loss: 0.6905 - val_accuracy: 0.5113
Epoch 2/15
192/192 - 16s - loss: 0.6007 - accuracy: 0.6675 - val_loss: 0.4910 - val_accuracy: 0.7645
Epoch 3/15
192/192 - 16s - loss: 0.3279 - accuracy: 0.8592 - val_loss: 0.48

...

Epoch 8/15
192/192 - 16s - loss: 0.0098 - accuracy: 0.9990 - val_loss: 1.2233 - val_accuracy: 0.7326
Epoch 9/15
192/192 - 16s - loss: 0.0131 - accuracy: 0.9967 - val_loss: 1.2439 - val_accuracy: 0.7383
Epoch 10/15
192/192 - 16s - loss: 0.0213 - accuracy: 0.9937 - val_loss: 1.3936 - val_accuracy: 0.7167
Epoch 11/15
192/192 - 16s - loss: 0.0212 - accuracy: 0.9945 - val_loss: 1.4981 - val_accuracy: 0.7073
Epoch 12/15
192/192 - 16s - loss: 0.0139 - accuracy: 0.9965 - val_loss: 1.5231 - val_accuracy: 0.7129
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 75.51594972610474

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  79.287720  74.976569  76.266414  76.454031  77.110696   
1       relu       2  80.693531  75.913775  76.735461  77.392119  75.609756   
2       relu       3  76.382381  77.038425  79.737335  76.078796  77.861166   
3       relu       4  77.507031  78.163075  77.016884  77.

Epoch 8/15
192/192 - 23s - loss: 0.0074 - accuracy: 0.9994 - val_loss: 1.1070 - val_accuracy: 0.7552
Epoch 9/15
192/192 - 24s - loss: 0.0057 - accuracy: 0.9993 - val_loss: 1.1825 - val_accuracy: 0.7514
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 76.73546075820923
Epoch 1/15
192/192 - 22s - loss: 0.6519 - accuracy: 0.5950 - val_loss: 0.5498 - val_accuracy: 0.7214
Epoch 2/15
192/192 - 21s - loss: 0.3532 - accuracy: 0.8567 - val_loss: 0.5321 - val_accuracy: 0.7636
Epoch 3/15
192/192 - 25s - loss: 0.1017 - accuracy: 0.9737 - val_loss: 0.6841 - val_accuracy: 0.7589
Epoch 4/15
192/192 - 19s - loss: 0.0351 - accuracy: 0.9930 - val_loss: 0.9020 - val_accuracy: 0.7542
Epoch 5/15
192/192 - 20s - loss: 0.0168 - accuracy: 0.9979 - val_loss: 1.0098 - val_accuracy: 0.7298
Epoch 6/15
192/192 - 23s - loss: 0.0110 - accuracy: 0.9982 - val_loss: 1.1222 - val_accuracy: 0.7355
Epoch 7/15
192/192 - 20s - loss: 0.0087 - accuracy: 0.9990 - val_loss: 1.14

Epoch 6/15
192/192 - 17s - loss: 0.0134 - accuracy: 0.9992 - val_loss: 1.0433 - val_accuracy: 0.7477
Epoch 7/15
192/192 - 17s - loss: 0.0097 - accuracy: 0.9995 - val_loss: 1.1137 - val_accuracy: 0.7448
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 77.95497179031372
Epoch 1/15
192/192 - 19s - loss: 0.6436 - accuracy: 0.6138 - val_loss: 0.5294 - val_accuracy: 0.7448
Epoch 2/15
192/192 - 18s - loss: 0.3593 - accuracy: 0.8535 - val_loss: 0.5022 - val_accuracy: 0.7505
Epoch 3/15
192/192 - 18s - loss: 0.1283 - accuracy: 0.9655 - val_loss: 0.6302 - val_accuracy: 0.7580
Epoch 4/15
192/192 - 20s - loss: 0.0453 - accuracy: 0.9922 - val_loss: 0.7932 - val_accuracy: 0.7580
Epoch 5/15
192/192 - 19s - loss: 0.0223 - accuracy: 0.9977 - val_loss: 0.9260 - val_accuracy: 0.7542
Epoch 6/15
192/192 - 21s - loss: 0.0148 - accuracy: 0.9990 - val_loss: 1.0170 - val_accuracy: 0.7448
Epoch 7/15
192/192 - 21s - loss: 0.0114 - accuracy: 0.9991 - val_loss: 1.14

Epoch 1/15
192/192 - 21s - loss: 0.6585 - accuracy: 0.5895 - val_loss: 0.5616 - val_accuracy: 0.7123
Epoch 2/15
192/192 - 17s - loss: 0.3521 - accuracy: 0.8586 - val_loss: 0.5430 - val_accuracy: 0.7338
Epoch 3/15
192/192 - 17s - loss: 0.0946 - accuracy: 0.9735 - val_loss: 0.7093 - val_accuracy: 0.7320
Epoch 4/15
192/192 - 17s - loss: 0.0338 - accuracy: 0.9940 - val_loss: 0.9027 - val_accuracy: 0.7310
Epoch 5/15
192/192 - 17s - loss: 0.0163 - accuracy: 0.9978 - val_loss: 1.0066 - val_accuracy: 0.7470
Epoch 6/15
192/192 - 19s - loss: 0.0105 - accuracy: 0.9989 - val_loss: 1.1533 - val_accuracy: 0.7338
Epoch 7/15
192/192 - 18s - loss: 0.0077 - accuracy: 0.9994 - val_loss: 1.2036 - val_accuracy: 0.7385
Epoch 8/15
192/192 - 18s - loss: 0.0097 - accuracy: 0.9980 - val_loss: 1.3581 - val_accuracy: 0.7301
Epoch 9/15
192/192 - 19s - loss: 0.0107 - accuracy: 0.9976 - val_loss: 1.4839 - val_accuracy: 0.7057
Epoch 10/15
192/192 - 19s - loss: 0.0112 - accuracy: 0.9976 - val_loss: 1.5686 - val_accura

Epoch 4/15
192/192 - 18s - loss: 0.0321 - accuracy: 0.9951 - val_loss: 0.8380 - val_accuracy: 0.7598
Epoch 5/15
192/192 - 16s - loss: 0.0139 - accuracy: 0.9990 - val_loss: 0.9770 - val_accuracy: 0.7627
Epoch 6/15
192/192 - 20s - loss: 0.0102 - accuracy: 0.9991 - val_loss: 1.0564 - val_accuracy: 0.7608
Epoch 7/15
192/192 - 20s - loss: 0.0077 - accuracy: 0.9994 - val_loss: 1.0899 - val_accuracy: 0.7664
Epoch 8/15
192/192 - 20s - loss: 0.0069 - accuracy: 0.9998 - val_loss: 1.1741 - val_accuracy: 0.7636
Epoch 9/15
192/192 - 23s - loss: 0.0053 - accuracy: 0.9998 - val_loss: 1.2369 - val_accuracy: 0.7598
Epoch 10/15
192/192 - 21s - loss: 0.0045 - accuracy: 0.9996 - val_loss: 1.2867 - val_accuracy: 0.7664
Epoch 11/15
192/192 - 22s - loss: 0.0035 - accuracy: 0.9997 - val_loss: 1.3412 - val_accuracy: 0.7655
Epoch 12/15
192/192 - 21s - loss: 0.0036 - accuracy: 0.9998 - val_loss: 1.3860 - val_accuracy: 0.7664
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test

## Summary

In [15]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
3,relu,4,77.507031,78.163075,77.016884,77.110696,77.954972,78.705442,76.172608,77.954972,76.923078,78.048778,77.555754
1,relu,2,80.693531,75.913775,76.735461,77.392119,75.609756,75.891185,77.298313,77.579737,77.392119,77.392119,77.189811
2,relu,3,76.382381,77.038425,79.737335,76.078796,77.861166,77.016884,76.829267,76.923078,76.641649,76.735461,77.124444
4,relu,5,77.132148,74.507964,75.328332,75.515950,75.891185,76.454031,79.643530,78.517824,76.735461,76.923078,76.664950
0,relu,1,79.287720,74.976569,76.266414,76.454031,77.110696,76.172608,76.172608,77.954972,76.923078,74.108815,76.542751
5,relu,6,74.976569,77.132148,76.266414,77.110696,77.204502,77.579737,76.641649,76.454031,75.515950,76.266414,76.514811
8,tanh,3,75.257730,74.882847,77.110696,77.110696,76.172608,76.078796,76.735461,76.454031,76.454031,75.515950,76.177285
7,tanh,2,76.288658,76.194942,76.923078,76.078796,74.859285,75.422138,76.547843,76.454031,75.515950,76.078796,76.036352
10,tanh,5,76.382381,77.132148,77.954972,75.797373,74.765480,77.016884,74.671668,73.358351,75.046903,77.954972,76.008113
6,tanh,1,74.039364,77.413309,77.204502,77.204502,76.641649,76.360226,74.953097,74.953097,74.765480,74.765480,75.830070


In [16]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,77.555754
tanh,76.177285


In [17]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_MR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 16448 words present from 18760 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [16]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.66467028, -0.27677347,  1.67170523, ...,  0.79809984,
        -0.02027692, -1.22025962],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## CNN Model

In [24]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [25]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(vocab_size, 300))
model_0.summary()

Model: "sequential_121"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_121 (Embedding)    (None, 100, 300)          5381700   
_________________________________________________________________
conv1d_121 (Conv1D)          (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_121 (MaxPoolin (None, 49, 100)           0         
_________________________________________________________________
flatten_121 (Flatten)        (None, 4900)              0         
_________________________________________________________________
dropout_242 (Dropout)        (None, 4900)              0         
_________________________________________________________________
dense_242 (Dense)            (None, 10)                49010     
_________________________________________________________________
dropout_243 (Dropout)        (None, 10)             

## Train and Test the Model

In [26]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [27]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=30, verbose=0, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 71.79006338119507
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 77.60074734687805
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 75.42213797569275
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 70.2626645565033
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 73.82739186286926
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 74.01500940322876
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 75.89118480682373
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 76.45403146743774
Restoring model weights from the end of the best epoch.
Epoch 000

Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Test Accuracy: 74.78913068771362
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Test Accuracy: 75.16401410102844
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 72.88930416107178
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 62.57035732269287
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Test Accuracy: 74.76547956466675
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Test Accuracy: 74.85928535461426
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 70.63789963722229
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Test Accuracy: 71.57598733901978
Restoring model weights from the end of the best epoch.
Epoch 00

## Summary

In [28]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,71.790063,77.600747,75.422138,70.262665,73.827392,74.015009,75.891185,76.454031,74.765480,76.454031,74.648274
1,relu,2,70.009375,72.727275,72.045028,75.891185,75.703567,71.575987,74.765480,76.547843,76.547843,74.108815,73.992240
4,relu,5,73.851919,74.226803,72.701687,75.984991,70.731705,68.667918,71.013135,73.452157,72.889304,72.514069,72.603369
6,relu,7,73.477036,69.447047,74.671668,73.545969,72.514069,72.889304,70.731705,71.575987,70.825517,73.545969,72.322427
3,relu,4,68.416119,71.040303,73.358351,64.821762,75.515950,76.266414,68.292683,73.921204,75.328332,74.484050,72.144517
2,relu,3,71.040303,78.069353,64.165103,72.326452,71.200752,76.078796,70.919323,75.328332,68.667918,69.043154,71.683949
5,relu,6,74.789131,75.164014,72.889304,62.570357,74.765480,74.859285,70.637900,71.575987,64.352721,69.230771,71.083495
7,relu,8,68.134958,71.508902,73.452157,72.232646,72.420263,51.125705,68.386489,73.545969,71.951222,70.637900,69.339621


In [29]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,74.648274


In [30]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_MR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [17]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [18]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(vocab_size, 300))
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          5628000   
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 4900)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4900)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                49010     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0

## Train and Test the Model

In [25]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [26]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=30, verbose=2, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/30
192/192 - 18s - loss: 0.6946 - accuracy: 0.5120 - val_loss: 0.6906 - val_accuracy: 0.5530
Epoch 2/30
192/192 - 16s - loss: 0.6845 - accuracy: 0.5534 - val_loss: 0.6772 - val_accuracy: 0.6120
Epoch 3/30
192/192 - 17s - loss: 0.6164 - accuracy: 0.6567 - val_loss: 0.5376 - val_accuracy: 0.7460
Epoch 4/30
192/192 - 17s - loss: 0.4753 - accuracy: 0.7490 - val_loss: 0.4558 - val_accuracy: 0.7929
Epoch 5/30
192/192 - 16s - loss: 0.3518 - accuracy: 0.8123 - val_loss: 0.4729 - val_accuracy: 0.7769
Epoch 6/30
192/192 - 17s - loss: 0.2467 - accuracy: 0.8776 - val_loss: 0.5452 - val_accuracy: 0.7854
Epoch 7/30
192/192 - 18s - loss: 0.1701 - accuracy: 0.9129 - val_loss: 0.7076 - val_accuracy: 0.7741
Epoch 8/30
192/192 - 17s - loss: 0.1313 - accuracy: 0.9553 - val_loss: 0.7801 - val_accuracy: 0.7563
Epoch 9/30
192/192 - 17s - loss: 0.1159 - accuracy: 0.9622 - val_loss: 0.9248 - val_accuracy: 0.7582
Epoch 10/30
192/192 - 18s - loss: 0.0946 - accuracy: 0.9694 - val_loss: 1.0613 - val_accura

Test Accuracy: 76.64164900779724
Epoch 1/30
192/192 - 17s - loss: 0.6990 - accuracy: 0.4949 - val_loss: 0.6931 - val_accuracy: 0.5028
Epoch 2/30
192/192 - 16s - loss: 0.6919 - accuracy: 0.5141 - val_loss: 0.6852 - val_accuracy: 0.5544
Epoch 3/30
192/192 - 16s - loss: 0.6566 - accuracy: 0.5874 - val_loss: 0.5716 - val_accuracy: 0.7439
Epoch 4/30
192/192 - 16s - loss: 0.5202 - accuracy: 0.7551 - val_loss: 0.4802 - val_accuracy: 0.7871
Epoch 5/30
192/192 - 16s - loss: 0.3652 - accuracy: 0.8547 - val_loss: 0.4625 - val_accuracy: 0.7805
Epoch 6/30
192/192 - 16s - loss: 0.2437 - accuracy: 0.9087 - val_loss: 0.5327 - val_accuracy: 0.7852
Epoch 7/30
192/192 - 16s - loss: 0.1621 - accuracy: 0.9401 - val_loss: 0.6945 - val_accuracy: 0.7777
Epoch 8/30
192/192 - 16s - loss: 0.1147 - accuracy: 0.9565 - val_loss: 0.8072 - val_accuracy: 0.7767
Epoch 9/30
192/192 - 16s - loss: 0.0957 - accuracy: 0.9626 - val_loss: 0.9993 - val_accuracy: 0.7598
Epoch 10/30
192/192 - 16s - loss: 0.0758 - accuracy: 0.969

Epoch 7/30
192/192 - 16s - loss: 0.2661 - accuracy: 0.8843 - val_loss: 0.5956 - val_accuracy: 0.7713
Epoch 8/30
192/192 - 16s - loss: 0.2005 - accuracy: 0.9263 - val_loss: 0.7833 - val_accuracy: 0.7451
Epoch 9/30
192/192 - 17s - loss: 0.1616 - accuracy: 0.9397 - val_loss: 1.0419 - val_accuracy: 0.7423
Epoch 10/30
192/192 - 17s - loss: 0.1277 - accuracy: 0.9489 - val_loss: 1.2405 - val_accuracy: 0.7479
Epoch 11/30
192/192 - 18s - loss: 0.1067 - accuracy: 0.9536 - val_loss: 1.4297 - val_accuracy: 0.7460
Epoch 12/30
192/192 - 17s - loss: 0.0841 - accuracy: 0.9632 - val_loss: 1.6050 - val_accuracy: 0.7395
Epoch 13/30
192/192 - 16s - loss: 0.0840 - accuracy: 0.9604 - val_loss: 1.8796 - val_accuracy: 0.7404
Epoch 14/30
192/192 - 16s - loss: 0.0728 - accuracy: 0.9670 - val_loss: 1.8948 - val_accuracy: 0.7395
Epoch 15/30
192/192 - 16s - loss: 0.0714 - accuracy: 0.9665 - val_loss: 2.2123 - val_accuracy: 0.7198
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
T

Epoch 2/30
192/192 - 16s - loss: 0.6863 - accuracy: 0.5505 - val_loss: 0.6791 - val_accuracy: 0.5797
Epoch 3/30
192/192 - 16s - loss: 0.6211 - accuracy: 0.6469 - val_loss: 0.5416 - val_accuracy: 0.7430
Epoch 4/30
192/192 - 16s - loss: 0.4830 - accuracy: 0.7833 - val_loss: 0.4972 - val_accuracy: 0.7580
Epoch 5/30
192/192 - 16s - loss: 0.3744 - accuracy: 0.8400 - val_loss: 0.5389 - val_accuracy: 0.7542
Epoch 6/30
192/192 - 16s - loss: 0.2855 - accuracy: 0.8820 - val_loss: 0.5783 - val_accuracy: 0.7477
Epoch 7/30
192/192 - 16s - loss: 0.1979 - accuracy: 0.9341 - val_loss: 0.7927 - val_accuracy: 0.7505
Epoch 8/30
192/192 - 16s - loss: 0.1295 - accuracy: 0.9603 - val_loss: 0.9370 - val_accuracy: 0.7430
Epoch 9/30
192/192 - 16s - loss: 0.0945 - accuracy: 0.9730 - val_loss: 1.1834 - val_accuracy: 0.7486
Epoch 10/30
192/192 - 16s - loss: 0.0708 - accuracy: 0.9833 - val_loss: 1.3472 - val_accuracy: 0.7242
Epoch 11/30
192/192 - 16s - loss: 0.0718 - accuracy: 0.9799 - val_loss: 1.3766 - val_accur

...

Epoch 9/30
192/192 - 17s - loss: 0.6898 - accuracy: 0.5128 - val_loss: 0.6870 - val_accuracy: 0.5441
Epoch 10/30
192/192 - 19s - loss: 0.6758 - accuracy: 0.5472 - val_loss: 0.6420 - val_accuracy: 0.6417
Epoch 11/30
192/192 - 18s - loss: 0.6089 - accuracy: 0.6100 - val_loss: 0.5603 - val_accuracy: 0.7270
Epoch 12/30
192/192 - 18s - loss: 0.5385 - accuracy: 0.6633 - val_loss: 0.4993 - val_accuracy: 0.7720
Epoch 13/30
192/192 - 18s - loss: 0.5023 - accuracy: 0.6806 - val_loss: 0.4900 - val_accuracy: 0.7674
Epoch 14/30
192/192 - 18s - loss: 0.4755 - accuracy: 0.6941 - val_loss: 0.4801 - val_accuracy: 0.7674
Epoch 15/30
192/192 - 17s - loss: 0.4351 - accuracy: 0.7284 - val_loss: 0.5313 - val_accuracy: 0.7692
Epoch 16/30
192/192 - 16s - loss: 0.3222 - accuracy: 0.8246 - val_loss: 0.6996 - val_accuracy: 0.7795
Epoch 17/30
192/192 - 20s - loss: 0.2414 - accuracy: 0.8712 - val_loss: 0.6946 - val_accuracy: 0.7739
Epoch 18/30
192/192 - 18s - loss: 0.2019 - accuracy: 0.8934 - val_loss: 0.7654 - va

192/192 - 26s - loss: 0.6993 - accuracy: 0.5031 - val_loss: 0.6931 - val_accuracy: 0.5178
Epoch 2/30
192/192 - 21s - loss: 0.6938 - accuracy: 0.4966 - val_loss: 0.6931 - val_accuracy: 0.5178
Epoch 3/30
192/192 - 18s - loss: 0.6935 - accuracy: 0.4949 - val_loss: 0.6932 - val_accuracy: 0.4822
Epoch 4/30
192/192 - 18s - loss: 0.6939 - accuracy: 0.4997 - val_loss: 0.6932 - val_accuracy: 0.4822
Epoch 5/30
192/192 - 17s - loss: 0.6936 - accuracy: 0.4926 - val_loss: 0.6936 - val_accuracy: 0.4822
Epoch 6/30
192/192 - 17s - loss: 0.6932 - accuracy: 0.4979 - val_loss: 0.6932 - val_accuracy: 0.4822
Epoch 7/30
192/192 - 17s - loss: 0.6930 - accuracy: 0.5019 - val_loss: 0.6933 - val_accuracy: 0.4822
Epoch 8/30
192/192 - 18s - loss: 0.6931 - accuracy: 0.5020 - val_loss: 0.6933 - val_accuracy: 0.4822
Epoch 9/30
192/192 - 17s - loss: 0.6931 - accuracy: 0.5020 - val_loss: 0.6933 - val_accuracy: 0.4822
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 51.

Epoch 1/30
192/192 - 28s - loss: 0.6984 - accuracy: 0.4961 - val_loss: 0.6930 - val_accuracy: 0.5005
Epoch 2/30
192/192 - 27s - loss: 0.6943 - accuracy: 0.5061 - val_loss: 0.6924 - val_accuracy: 0.5173
Epoch 3/30
192/192 - 22s - loss: 0.6914 - accuracy: 0.5142 - val_loss: 0.6907 - val_accuracy: 0.5211
Epoch 4/30
192/192 - 21s - loss: 0.6878 - accuracy: 0.5185 - val_loss: 0.6865 - val_accuracy: 0.5230
Epoch 5/30
192/192 - 21s - loss: 0.6853 - accuracy: 0.5227 - val_loss: 0.6817 - val_accuracy: 0.5323
Epoch 6/30
192/192 - 21s - loss: 0.6776 - accuracy: 0.5406 - val_loss: 0.6805 - val_accuracy: 0.5380
Epoch 7/30
192/192 - 21s - loss: 0.6644 - accuracy: 0.5611 - val_loss: 0.6715 - val_accuracy: 0.5689
Epoch 8/30
192/192 - 21s - loss: 0.6409 - accuracy: 0.6077 - val_loss: 0.6242 - val_accuracy: 0.6532
Epoch 9/30
192/192 - 21s - loss: 0.5768 - accuracy: 0.6642 - val_loss: 0.6666 - val_accuracy: 0.7301
Epoch 10/30
192/192 - 21s - loss: 0.5018 - accuracy: 0.7381 - val_loss: 0.6700 - val_accura

Epoch 5/30
192/192 - 20s - loss: 0.4068 - accuracy: 0.8168 - val_loss: 0.4810 - val_accuracy: 0.7777
Epoch 6/30
192/192 - 18s - loss: 0.3166 - accuracy: 0.8777 - val_loss: 0.5219 - val_accuracy: 0.7730
Epoch 7/30
192/192 - 17s - loss: 0.2414 - accuracy: 0.9184 - val_loss: 0.5574 - val_accuracy: 0.7814
Epoch 8/30
192/192 - 19s - loss: 0.1984 - accuracy: 0.9402 - val_loss: 0.6538 - val_accuracy: 0.7805
Epoch 9/30
192/192 - 19s - loss: 0.1576 - accuracy: 0.9544 - val_loss: 0.8572 - val_accuracy: 0.7711
Epoch 10/30
192/192 - 21s - loss: 0.1279 - accuracy: 0.9644 - val_loss: 1.0098 - val_accuracy: 0.7702
Epoch 11/30
192/192 - 22s - loss: 0.1057 - accuracy: 0.9756 - val_loss: 1.1097 - val_accuracy: 0.7683
Epoch 12/30
192/192 - 19s - loss: 0.0996 - accuracy: 0.9753 - val_loss: 1.1291 - val_accuracy: 0.7655
Epoch 13/30
192/192 - 21s - loss: 0.0863 - accuracy: 0.9789 - val_loss: 1.4305 - val_accuracy: 0.7758
Epoch 14/30
192/192 - 18s - loss: 0.0791 - accuracy: 0.9804 - val_loss: 1.6727 - val_ac

Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Test Accuracy: 75.14071464538574
Epoch 1/30
192/192 - 20s - loss: 0.7001 - accuracy: 0.5082 - val_loss: 0.6938 - val_accuracy: 0.4681
Epoch 2/30
192/192 - 18s - loss: 0.6937 - accuracy: 0.5077 - val_loss: 0.6933 - val_accuracy: 0.4775
Epoch 3/30
192/192 - 19s - loss: 0.6928 - accuracy: 0.5040 - val_loss: 0.6938 - val_accuracy: 0.4653
Epoch 4/30
192/192 - 17s - loss: 0.6926 - accuracy: 0.5051 - val_loss: 0.6937 - val_accuracy: 0.4672
Epoch 5/30
192/192 - 19s - loss: 0.6933 - accuracy: 0.5036 - val_loss: 0.6935 - val_accuracy: 0.4653
Epoch 6/30
192/192 - 18s - loss: 0.6929 - accuracy: 0.5048 - val_loss: 0.6932 - val_accuracy: 0.4737
Epoch 7/30
192/192 - 20s - loss: 0.6922 - accuracy: 0.5101 - val_loss: 0.6935 - val_accuracy: 0.4719
Epoch 8/30
192/192 - 19s - loss: 0.6925 - accuracy: 0.5063 - val_loss: 0.6946 - val_accuracy: 0.4644
Epoch 9/30
192/192 - 19s - loss: 0.6931 - accuracy: 0.5041 - val_loss: 0.69

## Summary

In [27]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
1,relu,2,77.132148,70.009375,80.112571,75.328332,77.392119,78.330207,75.797373,78.142589,76.454031,79.455912,76.815466
3,relu,4,76.476103,77.319586,78.611630,77.485931,77.392119,75.891185,79.362100,70.637900,77.767354,75.140715,76.608462
0,relu,1,79.287720,76.194942,78.893059,72.983116,76.641649,78.705442,72.983116,76.360226,77.579737,75.422138,76.505114
4,relu,5,74.789131,78.537959,75.422138,78.517824,77.954972,72.607881,75.422138,74.296433,76.735461,77.204502,76.148844
2,relu,3,77.788192,76.850986,74.671668,74.953097,73.921204,76.829267,76.923078,79.362100,75.422138,74.484050,76.120578
6,relu,7,72.258669,76.476103,78.048778,76.172608,72.795498,77.298313,51.782364,76.078796,77.861166,80.487806,73.926010
7,relu,8,77.038425,76.101220,79.080677,72.983116,78.142589,74.671668,78.986865,76.360226,75.140715,47.748592,73.625409
5,relu,6,78.631675,75.726336,50.093806,76.172608,76.923078,72.420263,75.046903,75.797373,76.454031,76.547843,73.381392


In [28]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_MR_3.xlsx', sheet_name='dynamic')